In [ ]:
import json
from pathlib import Path
import openai
import re
import math
import numpy as np
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import time

from utils import extract_numbers,find_first_empty_index,process_scientific_notation,extract_numeric_answer
openai.api_key  = "YOUR OPENAI API KEY"

from sentence_transformers import SentenceTransformer

model_sentence = SentenceTransformer("all-MiniLM-L6-v2")

In [2]:
# filename = 'PSA_OpenBench_Alpha.json'
# filename = 'PSA_OpenBench_Beta.json'
filename = 'PSA_OpenBench_Gamma.json'
# filename = 'PSA_PrivateBench_4.json'
# filename = 'PSA_PrivateBench_2.json'
# filename = 'PSA_PrivateBench_3.json'
# filename = 'PSA_PrivateBench_1.json'
data = json.loads(Path(filename).read_text())

In [3]:
def get_embedding(text):
    return model_sentence.encode(text)

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

### Example question

In [4]:
msg = '''The Viking Mars orbiter probe was placed in a circular orbit 17,000 km above the Martian surface. Doppler measurements of the transmitted signals from the probe indicated that it was at an orbital velocity, \( V_V \), of 1.46 km/s. The diameter of Mars, \( d_M \), is 6,770 km and the radius of the Mars orbit about the Sun, \( a_M \), is 1.524 AU. Note: 1 AU = 1.495 × 10^8 km; 1 year = 3.156 × 10^7 s. Calculate the mass of Mars, \( M_M \), in terms of the mass of the Sun, \( M_S \). Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).'''

<>:1: SyntaxWarning: invalid escape sequence '\('
<>:1: SyntaxWarning: invalid escape sequence '\('
/tmp/ipykernel_209012/4238521483.py:1: SyntaxWarning: invalid escape sequence '\('
  msg = '''The Viking Mars orbiter probe was placed in a circular orbit 17,000 km above the Martian surface. Doppler measurements of the transmitted signals from the probe indicated that it was at an orbital velocity, \( V_V \), of 1.46 km/s. The diameter of Mars, \( d_M \), is 6,770 km and the radius of the Mars orbit about the Sun, \( a_M \), is 1.524 AU. Note: 1 AU = 1.495 × 10^8 km; 1 year = 3.156 × 10^7 s. Calculate the mass of Mars, \( M_M \), in terms of the mass of the Sun, \( M_S \). Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).'''


In [5]:
# Load the model and tokenizer
model = AutoModelForCausalLM.from_pretrained("AstroMLab/AstroSage-8b", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("AstroMLab/AstroSage-8b")

# Function to generate a response
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        # max_new_tokens=256,
        # max_length=2024,
        max_length=6000,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )
    response = outputs[0][inputs['input_ids'].shape[-1]:]
    decoded = tokenizer.decode(response, skip_special_tokens=True)

    return decoded

# Example usage
prompt = """
You are an expert in general astrophysics. Your task is to answer the following question:
What are the main components of a galaxy?
"""
response = generate_response(prompt)
print(response)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Galaxies are composed of several main components, including a central bulge, a disk, and a halo. The central bulge is typically composed of older, more massive stars, while the disk contains younger, less massive stars. The halo is a diffuse region surrounding the galaxy, containing older stars and dark matter. Additionally, galaxies may have spiral arms, bars, and other structures. The composition and distribution of these components can vary greatly between different types of galaxies. For example, spiral galaxies have a prominent disk and spiral arms, while elliptical galaxies have a more spherical shape with a central bulge. The halo is typically the largest component of a galaxy, but its exact composition and structure can be difficult to determine due to its diffuse nature. Overall, the main components of a galaxy work together to create a complex and dynamic system that is constantly evolving. The study of these components and their interactions is an active area of research in 

In [6]:
start_time = time.time()
for i in range(1):
    # messages = [
    #     {"role": "user", "content": msg}
    # ]
    # generated_text = pipe(
    #     messages,
    #     max_length = 2024
    # )
    reply = generate_response(msg)
    print(reply)
print(f'Elapsed time: {(time.time()-start_time)/1}')

 The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer is 0.107. The answer i

In [10]:
len(data[10]['Questions'])

2

In [ ]:
match_result = []  # Universal array of results
match_result = [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

for idx in range(58,65): #range(45):#range(7): #
    # idx = 2
    print("_____________")
    print("ID", idx)
    print("current_match_result: ", match_result)


    content_str = data[idx]['Content']
    questions_solutions = data[idx]['Questions']
    question_strs = [entry['Question'] for entry in questions_solutions]
    solution_strs = [entry['Solution'] for entry in questions_solutions]
    answer_strs = [entry['Answer'] for entry in questions_solutions]
    format_strs = [entry['Format'] for entry in questions_solutions]
    match_results_local = [[] for entry in questions_solutions]

    # print("content: ",content_str)

    correct_QAs = []
    for q_idx, question_str in enumerate(question_strs):
        print("______")
        print("q_idx: ", q_idx)
        format_str = format_strs[q_idx]
        solution_str = solution_strs[q_idx]
        answer_str = answer_strs[q_idx]
        print("Answer: ", answer_str)
        all_correct_QAs = " ".join(correct_QAs)
        msg = f"{all_correct_QAs} {content_str} {question_str} Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words)."

        print("msg", msg)
        
        # calling model
        reply = generate_response(msg)
        print("reply: ", reply)
        
        pattern = r'the answer is \s*(.*)|The answer is \s*(.*)' 
        match = re.search(pattern, reply)
        try:
            user_answer_str = match.group(0)
            print("user_answer_str: ", user_answer_str)
        except:
            user_answer_str = None

        if 'message' == format_str:
            print("MESSAGE")

            score = 0

            # embedding score
            score_embedding = cosine_similarity(get_embedding(reply), get_embedding(answer_str))
            print(f"similarity: {score_embedding}")
            
            # gpt score
            # gpt prompt
            message = f"""
            Compare the following reply with the expected answer and evaluate their alignment using these criteria:

            1. **Relevance:** Does the reply address the scientific goals and concepts mentioned in the expected answer?  
            2. **Completeness:** Does the reply cover the key points about the need for a deep periapsis and a large range of distances?  
            3. **Accuracy:** Is the reasoning in the reply correct and consistent with the scientific explanation?  

            Question: {question_str}  
            Expected Answer: {answer_str}  
            Reply: {reply}  

            After evaluating, provide a similarity score between 0 and 10, where:  
            - **10** means the reply perfectly aligns with the expected answer.  
            - **0** means the reply does not align at all.  

            Only return the numeric score as your final output.
            """

            # response part
            response = openai.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": message}],
                temperature=0,
            )
            gpt_similarity = response.choices[0].message.content
            score_gpt = float(gpt_similarity) / 10 

            # combining scores
            score = score_embedding*0.5 + score_gpt*0.5
            print(f"score_gpt: {score_gpt}")
            print(f"SCORE: {score}")
            
            if score > 0.6: # more than 60% affirmative
                match_results_local[q_idx] = 1
                extra_str = "For reference, a previous question: '" + content_str + question_str + " '  has answer " + reply
                correct_QAs.append(extra_str)
            else:
                match_results_local[q_idx] = 0 

            ## formated output
            print(f'LLM answer: {reply}')
            print(f'Reference answer: {answer_str}')
            print('True' if match_results_local[q_idx]==1 else 'False')
           
        if 'numeric' == format_str:
            print("NUMERIC")

            power = []
            try:
                if user_answer_str is None:
                    user_answer_str = ''
                    raise TypeError("user_answer_str cannot be None")
                user_answer_num_str = extract_numeric_answer(user_answer_str)
                user_answer_num = extract_numbers(user_answer_num_str)[0]
            except:
                # check if answer string has power expression
                user_answer_str, power = process_scientific_notation(user_answer_str)
                user_answer_num = None

            # when openai is able to handle the operation
            if not power:
                user_answer_num = user_answer_num
                answer_str_num = extract_numbers(answer_str)[0]
                match_results_local[q_idx] = 0
            # other cases
            else:
                try:
                    if power:
                        print("pow")
                        user_answer_pow = extract_numbers(user_answer_str.strip())[2]
                        user_answer_num = extract_numbers(user_answer_str.strip())[0] * 10**user_answer_pow
                        power = extract_numbers(answer_str)[1]
                        answer_str_num = extract_numbers(answer_str)[0] * 10**power

                        print("user_answer_num", user_answer_num)
                        print("answer_str_num", answer_str_num)

                    else:
                        print("no pow")
                        print(user_answer_str.strip())
                        user_answer_num = extract_numbers(user_answer_str.strip())[0]
                        answer_str_num = extract_numbers(answer_str)[0]

                        print("user_answer_num", user_answer_num)
                        print("answer_str_num", answer_str_num)

                except:
                    user_answer_num = None
                    answer_str_num = None
                    match_results_local[q_idx] = 0

            if user_answer_num is not None and user_answer_num != 0:
                
                margin = 0.1 + 0.01 * math.log(abs(user_answer_num))
                print("margin: ", margin)
                if answer_str_num == user_answer_num:
                    match_results_local[q_idx] = 1
                    extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
                    correct_QAs.append(extra_str)
                    # print("Exact Match")
                elif abs(answer_str_num - user_answer_num) / abs(user_answer_num) < abs(margin):
                    match_results_local[q_idx] = 1
                    extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
                    correct_QAs.append(extra_str)
                    # print("Rough Match")
                else:
                    match_results_local[q_idx] = 0
                    # print("No Match")
            
            ## formated output
            print(f'LLM answer: {user_answer_str}')
            print(f'Reference answer: {answer_str}')
            print('True' if match_results_local[q_idx]==1 else 'False')

    if match_results_local:
        match_results_local_new = [item for item in match_results_local if not (isinstance(item, list) and len(item) == 0)]
        print("local", match_results_local_new)
        match_result.extend(match_results_local_new)

# Print the universal match result array
print("Universal match result array:", match_result)


total_accuracy = sum(match_result) / len(match_result)

_____________
ID 58
current_match_result:  [0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
______
q_idx:  0
Answer:  -3791.73 km
msg  Given satellite parameters as spotted by the radar tracking station in Example 1:

- \( \rho \) = 7000 km
- Az = 40 degrees
- El = 45 degrees How do you calculate the \(\rho_S\) component

In [7]:
print(total_accuracy)

0.1605351170568562


## Specific Question Testbed

In [21]:
from pprint import pprint 
idx = 0
print("_____________")
print("Content ID", idx)

content_str = data[idx]['Content']
questions_solutions = data[idx]['Questions']
question_strs = [entry['Question'] for entry in questions_solutions]
solution_strs = [entry['Solution'] for entry in questions_solutions]
answer_strs = [entry['Answer'] for entry in questions_solutions]
format_strs = [entry['Format'] for entry in questions_solutions]
match_results_local = [[] for entry in questions_solutions]

print("Content: ",content_str)
print("Questions: ")
question_strs


_____________
Content ID 0
Content:  You are fortunate to be an astronaut sent out to explore planet X. This planet has a mass that is four times greater than the Earth and three times its radius.
Questions: 


['What is the formula for the gravitational force experienced by an object on Earth?',
 'How does the gravitational force on planet X compare to that on Earth? Please provide the answer as a decimal multiplier of the gravitational force on Earth.',
 'How does your weight on planet X compare to your weight on Earth? Please provide the answer in the form of a multiplier indicating how much more one would weigh on planet X compared to Earth.']

In [22]:
q_idx = 1
question_str = question_strs[q_idx]

correct_QAs = []
print("______")
print("q_idx: ", q_idx)
format_str = format_strs[q_idx]
solution_str = solution_strs[q_idx]
answer_str = answer_strs[q_idx]
print("Question: ", question_str)
print("Answer: ", answer_str)
all_correct_QAs = " ".join(correct_QAs)
msg = f"{all_correct_QAs} {content_str} {question_str} Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words)."
print('-----------------------------------------')
power = False

if r" \times 10^" in answer_str:
    power = True
    answer_str = answer_str.replace(r" \times 10^", "e")
    print("new_answer_str: ", answer_str)

print("msg", msg)
print('-----------------------------------------')

# calling model
message = client.messages.create(
    model="claude-3-5-sonnet-20241022",
    max_tokens=1000,
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": msg
                }
            ]
        }
    ]
)

reply = message.content[0].text
print("reply: ", reply)

pattern = r'the answer is \s*(.*)|The answer is \s*(.*)' 
match = re.search(pattern, reply)
try:
    user_answer_str = match.group(0)
    print("user_answer_str: ", user_answer_str)
    print('-----------------------------------------')
except:
    user_answer_str = None

if 'message' == format_str:
    print("MESSAGE")

    score = 0

    if user_answer_str is not None:
        print("similarity", cosine_similarity(get_embedding(user_answer_str), get_embedding(answer_str)))
        score += cosine_similarity(get_embedding(user_answer_str), get_embedding(answer_str))
    
    message = f"Please give a similarity score of a number between 0 and 10 for {user_answer_str} and {answer_str}. Only a number, nothing else."
    # response part
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": message}],
        temperature=0,
    )
    gpt_similarity = response.choices[0].message.content
    gpt_score = float(gpt_similarity) * 0.05
    score += gpt_score
    print("gpt_score: ", gpt_score)
    print("SCORE: ", score)
    if score > 1:
        match_results_local[q_idx] = 1
        extra_str = "For reference, a previous question: '" + content_str + question_str + " '  has answer " + str(user_answer_num)
        correct_QAs.append(extra_str)
    else:
        match_results_local[q_idx] = 0 
    

if 'numeric' == format_str:
    print("NUMERIC")

    try:
        if power:
            print("pow")
            user_answer_pow = extract_numbers(user_answer_str.strip())[2]
            user_answer_num = extract_numbers(user_answer_str.strip())[0] * 10**user_answer_pow
            power = extract_numbers(answer_str)[1]
            answer_str_num = extract_numbers(answer_str)[0] * 10**power

            print("user_answer_num", user_answer_num)
            print("answer_str_num", answer_str_num)

        else:
            print("no pow")
            print(user_answer_str.strip())
            user_answer_num = extract_numbers(user_answer_str.strip())[0]
            answer_str_num = extract_numbers(answer_str)[0]

            print("user_answer_num", user_answer_num)
            print("answer_str_num", answer_str_num)

    except:
        user_answer_num = None
        answer_str_num = None
        match_results_local[q_idx] = 0

    if user_answer_num is not None and user_answer_num != 0:
        
        margin = 0.1 + 0.01 * math.log(abs(user_answer_num))
        print("margin: ", margin)
        if answer_str_num == user_answer_num:
            match_results_local[q_idx] = 1
            extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
            correct_QAs.append(extra_str)
            # print("Exact Match")
        elif abs(answer_str_num - user_answer_num) / abs(user_answer_num) < margin:
            match_results_local[q_idx] = 1
            extra_str = "For reference, a previous question: " + question_str + ". It has answer " + str(user_answer_num)
            correct_QAs.append(extra_str)
            # print("Rough Match")
        else:
            match_results_local[q_idx] = 0
            # print("No Match")


______
q_idx:  1
Question:  How does the gravitational force on planet X compare to that on Earth? Please provide the answer as a decimal multiplier of the gravitational force on Earth.
Answer:  0.44 times the gravitational force on Earth
-----------------------------------------
msg  You are fortunate to be an astronaut sent out to explore planet X. This planet has a mass that is four times greater than the Earth and three times its radius. How does the gravitational force on planet X compare to that on Earth? Please provide the answer as a decimal multiplier of the gravitational force on Earth. Without any extra words, please specify the answer in the final sentence with this form: 'the answer is (answer)'. The answer should either be numeric or symbolic (not words).
-----------------------------------------
reply:  Let me solve this step by step:

1) The gravitational force (g) is proportional to Mass (M) and inversely proportional to radius (r) squared:
g ∝ M/r²

2) For Earth, we c

In [9]:
match_results_local

[[], 1, []]